In [19]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [20]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1740989816, 'localtime': '2025-03-03 00:16'}, 'current': {'last_updated_epoch': 1740989700, 'last_updated': '2025-03-03 00:15', 'temp_c': 10.3, 'temp_f': 50.5, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 15.2, 'wind_kph': 24.5, 'wind_degree': 294, 'wind_dir': 'WNW', 'pressure_mb': 1019.0, 'pressure_in': 30.1, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 71, 'cloud': 25, 'feelslike_c': 7.4, 'feelslike_f': 45.3, 'windchill_c': 6.3, 'windchill_f': 43.3, 'heatindex_c': 9.4, 'heatindex_f': 48.9, 'dewpoint_c': 6.5, 'dewpoint_f': 43.7, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 22.3, 'gust_kph': 36.0}}"}, {'url': 'https://weather.com/weather/today/l

In [25]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.2",
    temperature=0,
    # other params...
)

# model = ChatOllama(
#     model="deepseek-r1:32b",
#     temperature=0.3,
#     base_url = "http://10.1.4.136:11434/"
# )
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content


'How can I assist you today?'

In [26]:
#现在我们可以调用模型了。我们先用普通消息调用它，看看它如何响应。
# 我们可以同时查看字段content和tool_calls字段。
# ps:这个model需要支持tool的模型才可以
model_with_tools = model.bind_tools(tools)

response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current events'}, 'id': '7f148f43-8a57-4e6d-8ace-9cf87cd73424', 'type': 'tool_call'}]


In [27]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [28]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='258c5073-e665-401f-a4c9-1cf9446b5731'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-03-03T08:55:47.270813Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1274652334, 'load_duration': 30726292, 'prompt_eval_count': 188, 'prompt_eval_duration': 251000000, 'eval_count': 22, 'eval_duration': 991000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-160ce533-2c33-41f6-b787-26d5c5261838-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current events'}, 'id': '3f5ab38c-b381-469f-825f-dc8ab538f8da', 'type': 'tool_call'}], usage_metadata={'input_tokens': 188, 'output_tokens': 22, 'total_tokens': 210}),
 ToolMessage(content='[{"url": "https://www.nytimes.com/spotlight/learning-current-events", "content": "Current Events - The New York Times Resources for teaching about current events 

为了确切了解幕后发生了什么（并确保它没有调用工具），我们可以查看LangSmith 跟踪

现在让我们在一个应该调用该工具的示例上尝试一下

In [29]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='80919522-883e-4fa6-b5c8-4982bbb60ede'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-03-03T08:58:45.510073Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1763595500, 'load_duration': 31795125, 'prompt_eval_count': 193, 'prompt_eval_duration': 769000000, 'eval_count': 22, 'eval_duration': 961000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-f543c5ad-5be1-4d90-ae11-2efaf9d9637e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '0b963291-d8b4-4919-812d-7dd6fd245c19', 'type': 'tool_call'}], usage_metadata={'input_tokens': 193, 'output_tokens': 22, 'total_tokens': 215}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'Unite

# Streaming Messages
我们已经了解了如何调用代理.invoke来获得最终响应。如果代理执行多个步骤，这可能需要一段时间。为了显示中间进度，我们可以在消息发生时回传消息。

In [30]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (6497cee7-c644-4c6e-8cc5-08ae0774c1ed)
 Call ID: 6497cee7-c644-4c6e-8cc5-08ae0774c1ed
  Args:
    query: sf weather
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1740992856, 'localtime': '2025-03-03 01:07'}, 'current': {'last_updated_epoch': 1740992400, 'last_updated': '2025-03-03 01:00', 'temp_c': 10.2, 'temp_f': 50.4, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 15.